In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

stock_list = ["NVDA", "AAPL", "MSFT"] # Expandable
risk_free_rate = 0.05
threshold_iv_hv = 0.5  # 50% difference ( can be decreased for more opportunities ) 
mispricing_margin = 4.0  
min_price = 1.0
max_iv = 5.0
max_expiry_days = 180

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

stock_list = ["NVDA", "AAPL", "MSFT"] # Expandable
risk_free_rate = 0.05
threshold_iv_hv = 0.5  # 50% difference ( can be decreased for more opportunities ) 
mispricing_margin = 4.0  
min_price = 1.0
max_iv = 5.0
max_expiry_days = 180

In [4]:
def get_hv(ticker, days=30):
    data = yf.download(ticker, period=f"{days+1}d")["Close"]
    log_returns = np.log(data / data.shift(1)).dropna()
    daily_std = log_returns.std()
    annualized_hv = daily_std * np.sqrt(252)
    return annualized_hv


In [5]:
# Theoritical Prcing using Black-Scholes Model

def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call

In [6]:
def scan_stock(ticker):
    print(f"\n Scanning {ticker}...")
    stock = yf.Ticker(ticker)
    spot = stock.history(period="1d")["Close"].iloc[-1]
    hv =float(get_hv(ticker).iloc[0])

    # nearest 2 expiries
    trade_ideas = []

    for expiry in stock.options:
        options = stock.option_chain(expiry).calls
        T = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.today()).days / 365

        for _, row in options.iterrows():
            K = row["strike"]
            market_price = row["lastPrice"]
            IV = row["impliedVolatility"]
            if IV is None or market_price == 0:
                continue

            theo_price = bs_call_price(spot, K, T, risk_free_rate, IV)
            iv_diff = IV - hv

            # Our AND gate
            if(abs(float(iv_diff)) > threshold_iv_hv and abs(float(theo_price - market_price)) > mispricing_margin):
                    direction = "BUY" if theo_price > market_price else "SELL"
                    trade_ideas.append({
                    "ticker": ticker,
                    "expiry": expiry,
                    "strike": K,
                    "spot": round(spot, 2),
                    "IV": round(IV, 2),
                    "HV": round(hv, 2),
                    "market_price": market_price,
                    "theo_price": round(theo_price, 2),
                    "direction": direction
                })

    return trade_ideas

In [22]:
def scan_stock(ticker):
    print(f"\n Scanning {ticker}...")
    trade_ideas = []
    
    try:
        stock = yf.Ticker(ticker)
        spot = stock.history(period="1d")["Close"].iloc[-1]
        hv = float(get_hv(ticker))

        for expiry in stock.options:
            expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
            days_to_expiry = (expiry_date - datetime.today()).days
            if days_to_expiry <= 0 or days_to_expiry > max_expiry_days:
                continue

            options = stock.option_chain(expiry).calls
            T = days_to_expiry / 365

            for _, row in options.iterrows():
                K = row["strike"]
                market_price = row["lastPrice"]
                IV = row["impliedVolatility"]

                if IV is None or market_price is None:
                    continue

                if market_price < min_price or IV > max_iv:
                    continue

                if K < 0.1 * spot or K > 2.0 * spot:
                    continue

                theo_price = bs_call_price(spot, K, T, risk_free_rate, IV)
                mispricing = abs(theo_price - market_price)
                iv_diff = abs(IV - hv)

                if iv_diff > threshold_iv_hv and mispricing > mispricing_margin:
                    direction = "BUY" if theo_price > market_price else "SELL"
                    trade_ideas.append({
                        "ticker": ticker,
                        "expiry": expiry,
                        "strike": K,
                        "spot": round(spot, 2),
                        "IV": round(IV, 2),
                        "HV": round(hv, 2),
                        "market_price": round(market_price, 2),
                        "theo_price": round(theo_price, 2),
                        "mispricing": round(mispricing, 2),
                        "direction": direction
                    })

    except Exception as e:
        print(f"Error with {ticker}: {e}")
    
    return sorted(trade_ideas, key=lambda x: x["mispricing"], reverse=True)

In [23]:
final_signals = []

for stock in stock_list:
    results = scan_stock(stock)
    final_signals.extend(results)

df = pd.DataFrame(final_signals)
print(df)



 Scanning NVDA...


[*********************100%***********************]  1 of 1 completed
/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))
/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/3040398318.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))



 Scanning AAPL...


[*********************100%***********************]  1 of 1 completed
/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))



 Scanning MSFT...


[*********************100%***********************]  1 of 1 completed
/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))


   ticker      expiry  strike    spot    IV    HV  market_price  theo_price  \
0    NVDA  2025-06-20    15.5  101.49  0.00  0.79        124.61       86.12   
1    NVDA  2025-06-20    33.5  101.49  0.00  0.79         86.48       68.27   
2    NVDA  2025-06-20    29.5  101.49  1.43  0.79         58.90       72.43   
3    NVDA  2025-06-20    19.5  101.49  1.74  0.79         93.60       82.26   
4    NVDA  2025-06-20    22.5  101.49  1.75  0.79         90.60       79.39   
5    NVDA  2025-09-19    54.0  101.49  0.00  0.79         59.75       48.61   
6    NVDA  2025-06-20    16.0  101.49  1.90  0.79         96.76       85.71   
7    NVDA  2025-06-20    28.5  101.49  1.53  0.79         84.50       73.49   
8    NVDA  2025-05-16    35.0  101.49  1.64  0.79         77.25       66.69   
9    NVDA  2025-05-23    35.0  101.49  1.55  0.79         77.00       66.77   
10   NVDA  2025-06-20    27.0  101.49  1.44  0.79         85.00       74.85   
11   NVDA  2025-06-20    21.0  101.49  1.57  0.79   

In [24]:
final_signals = []
for stock in stock_list:
    final_signals.extend(scan_stock(stock))

df = pd.DataFrame(final_signals)
pd.set_option('display.max_rows', None)
print("\n Mispricing Signals :")
print(df.head(10))



[*********************100%***********************]  1 of 1 completed


 Scanning NVDA...



/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))
/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/3040398318.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
[*********************100%***********************]  1 of 1 completed


 Scanning AAPL...



/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))
[*********************100%***********************]  1 of 1 completed


 Scanning MSFT...



/var/folders/qt/d6c46fmx0qq17bly4bvw1lfw0000gn/T/ipykernel_26451/2805775286.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hv = float(get_hv(ticker))



 Mispricing Signals :
  ticker      expiry  strike    spot    IV    HV  market_price  theo_price  \
0   NVDA  2025-06-20    15.5  101.49  0.00  0.79        124.61       86.12   
1   NVDA  2025-06-20    33.5  101.49  0.00  0.79         86.48       68.27   
2   NVDA  2025-06-20    29.5  101.49  1.43  0.79         58.90       72.43   
3   NVDA  2025-06-20    19.5  101.49  1.74  0.79         93.60       82.26   
4   NVDA  2025-06-20    22.5  101.49  1.75  0.79         90.60       79.39   
5   NVDA  2025-09-19    54.0  101.49  0.00  0.79         59.75       48.61   
6   NVDA  2025-06-20    16.0  101.49  1.90  0.79         96.76       85.71   
7   NVDA  2025-06-20    28.5  101.49  1.53  0.79         84.50       73.49   
8   NVDA  2025-05-16    35.0  101.49  1.64  0.79         77.25       66.69   
9   NVDA  2025-05-23    35.0  101.49  1.55  0.79         77.00       66.77   

   mispricing direction  
0       38.49      SELL  
1       18.21      SELL  
2       13.53       BUY  
3       11.34 

In [25]:
#  Theoretical price is greater than market price 
mispriced_df = df[df['theo_price'] > df['market_price']]

# IV > HV (Implied Vol > Historical Vol)
mispriced_df = mispriced_df[mispriced_df['IV'] > mispriced_df['HV']]

# Final and filtered mispricing opportunities
print("Filtered Mispricing Signals:")
print(mispriced_df[['ticker', 'expiry', 'strike', 'spot', 'IV', 'HV', 'market_price', 'theo_price']])


Filtered Mispricing Signals:
   ticker      expiry  strike    spot    IV    HV  market_price  theo_price
2    NVDA  2025-06-20    29.5  101.49  1.43  0.79         58.90       72.43
12   NVDA  2025-06-20    17.5  101.49  1.75  0.79         75.00       84.21
15   NVDA  2025-06-20    28.0  101.49  1.46  0.79         65.00       73.90
17   NVDA  2025-06-20    26.5  101.49  1.57  0.79         67.40       75.43
18   NVDA  2025-06-20    32.5  101.49  1.42  0.79         61.75       69.56
20   NVDA  2025-06-20    25.5  101.49  1.56  0.79         69.00       76.38
21   NVDA  2025-07-18    20.0  101.49  1.55  0.79         74.83       81.93
22   NVDA  2025-06-20    11.5  101.49  2.09  0.79         83.15       90.13
23   NVDA  2025-06-20    17.0  101.49  1.61  0.79         77.80       84.66
24   NVDA  2025-06-20    12.5  101.49  2.00  0.79         82.45       89.14
25   NVDA  2025-06-20    10.5  101.49  2.17  0.79         84.65       91.13
28   NVDA  2025-06-20    12.0  101.49  2.13  0.79         8

In [26]:
def calculate_delta(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    if option_type == 'call':
        delta = norm.cdf(d1)
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
    else:
        raise ValueError("option_type must be 'call' or 'put'")
        
    return delta

In [27]:
from datetime import datetime

today = datetime.today()
mispriced_df["T"] = (pd.to_datetime(mispriced_df["expiry"]) - today).dt.days / 365


In [28]:
risk_free_rate = 0.05  

mispriced_df["delta"] = mispriced_df.apply(
    lambda row: calculate_delta(
        S=row["spot"],
        K=row["strike"],
        T=row["T"],
        r=risk_free_rate,
        sigma=row["IV"],
        option_type='call' # Call options used specifically, can be changed to puts though. 
    ),
    axis=1
)

In [29]:
print(mispriced_df[["ticker", "expiry", "strike", "spot", "IV", "T", "delta"]])


   ticker      expiry  strike    spot    IV         T     delta
2    NVDA  2025-06-20    29.5  101.49  1.43  0.169863  0.991926
12   NVDA  2025-06-20    17.5  101.49  1.75  0.169863  0.997519
15   NVDA  2025-06-20    28.0  101.49  1.46  0.169863  0.992957
17   NVDA  2025-06-20    26.5  101.49  1.57  0.169863  0.992065
18   NVDA  2025-06-20    32.5  101.49  1.42  0.169863  0.987866
20   NVDA  2025-06-20    25.5  101.49  1.56  0.169863  0.993487
21   NVDA  2025-07-18    20.0  101.49  1.55  0.246575  0.993983
22   NVDA  2025-06-20    11.5  101.49  2.09  0.169863  0.998504
23   NVDA  2025-06-20    17.0  101.49  1.61  0.169863  0.998806
24   NVDA  2025-06-20    12.5  101.49  2.00  0.169863  0.998477
25   NVDA  2025-06-20    10.5  101.49  2.17  0.169863  0.998620
28   NVDA  2025-06-20    12.0  101.49  2.13  0.169863  0.998016
30   NVDA  2025-06-20    15.0  101.49  1.91  0.169863  0.997695
31   NVDA  2025-06-20    13.5  101.49  1.93  0.169863  0.998382
32   NVDA  2025-06-20    27.5  101.49  1

In [30]:
capital = 1_000_000  # $1M

mispriced_df["capital_allocated"] = capital / len(mispriced_df)

# Number of contracts to buy  
mispriced_df["contracts"] = (mispriced_df["capital_allocated"] / (mispriced_df["market_price"] * 100)).astype(int)

# Hedging shares = contracts * 100 * delta
mispriced_df["hedge_shares"] = mispriced_df["contracts"] * 100 * mispriced_df["delta"]

# Total money used per position
mispriced_df["capital_used"] = mispriced_df["contracts"] * mispriced_df["market_price"] * 100


In [31]:
cols = ["ticker", "expiry", "strike", "spot", "market_price", "delta", "contracts", "hedge_shares", "capital_used"]
print(mispriced_df[cols])


   ticker      expiry  strike    spot  market_price     delta  contracts  \
2    NVDA  2025-06-20    29.5  101.49         58.90  0.991926          6   
12   NVDA  2025-06-20    17.5  101.49         75.00  0.997519          5   
15   NVDA  2025-06-20    28.0  101.49         65.00  0.992957          6   
17   NVDA  2025-06-20    26.5  101.49         67.40  0.992065          5   
18   NVDA  2025-06-20    32.5  101.49         61.75  0.987866          6   
20   NVDA  2025-06-20    25.5  101.49         69.00  0.993487          5   
21   NVDA  2025-07-18    20.0  101.49         74.83  0.993983          5   
22   NVDA  2025-06-20    11.5  101.49         83.15  0.998504          4   
23   NVDA  2025-06-20    17.0  101.49         77.80  0.998806          5   
24   NVDA  2025-06-20    12.5  101.49         82.45  0.998477          4   
25   NVDA  2025-06-20    10.5  101.49         84.65  0.998620          4   
28   NVDA  2025-06-20    12.0  101.49         83.65  0.998016          4   
30   NVDA  2

In [32]:
total_used = mispriced_df["capital_used"].sum()
total_hedge_shares = mispriced_df["hedge_shares"].sum()
net_portfolio_delta = mispriced_df["delta"].mul(mispriced_df["contracts"] * 100).sum()

print(f" Capital deployed: ${total_used:,.2f}")
print(f" Total hedge shares needed : {total_hedge_shares:,.2f}")
print(f" Net portfolio delta : {net_portfolio_delta:.2f}")


 Capital deployed: $879,412.00
 Total hedge shares needed : 9,880.50
 Net portfolio delta : 9880.50


In [33]:
mispriced_df["option_pnl"] = (mispriced_df["theo_price"] - mispriced_df["market_price"]) * mispriced_df["contracts"] * 100
total_pnl = mispriced_df["option_pnl"].sum()

print(mispriced_df[["ticker", "strike", "market_price", "theo_price", "contracts", "option_pnl"]])
print(f"\n Espimated profits if options converge to theoretical value : ${total_pnl:,.2f}")


   ticker  strike  market_price  theo_price  contracts  option_pnl
2    NVDA    29.5         58.90       72.43          6      8118.0
12   NVDA    17.5         75.00       84.21          5      4605.0
15   NVDA    28.0         65.00       73.90          6      5340.0
17   NVDA    26.5         67.40       75.43          5      4015.0
18   NVDA    32.5         61.75       69.56          6      4686.0
20   NVDA    25.5         69.00       76.38          5      3690.0
21   NVDA    20.0         74.83       81.93          5      3550.0
22   NVDA    11.5         83.15       90.13          4      2792.0
23   NVDA    17.0         77.80       84.66          5      3430.0
24   NVDA    12.5         82.45       89.14          4      2676.0
25   NVDA    10.5         84.65       91.13          4      2592.0
28   NVDA    12.0         83.65       89.66          4      2404.0
30   NVDA    15.0         81.30       86.68          4      2152.0
31   NVDA    13.5         82.90       88.15          4      21